#### Dependencies:

In [1]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import ollama
import os
import re
import hashlib
import base64
import json

### Functions:

In [2]:
def gen_hash(string):
    hash_obj = hashlib.sha256()
    hash_obj.update(string.encode('utf-8'))
    return hash_obj.hexdigest()

In [3]:
def get_json_value(json_data):
    try:
        if isinstance(json_data, str):
            json_data = json.loads(json_data)
        
        return next(iter(json_data.values()))
    except:
        return -2

### Directory:

In [4]:
# Caminhos para os diretórios das imagens

pleasant_p_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Pleasant (+3)"
pleasant_p_dir_name = "pleasant_p"

unpleasant_p_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Unpleasant (+3)"
unpleasant_p_dir_name = "unpleasant_p"

pleasant_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Pleasant"
pleasant_dir_name = "pleasant"

unpleasant_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Unpleasant"
unpleasant_dir_name = "unpleasant"

pleasant_n_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Pleasant (-3)"
pleasant_n_dir_name = "pleasant_n"

unpleasant_n_dir = "/home/jerre/git/Stable_Diffusion_IC/images/Unpleasant (-3)"
unpleasant_n_dir_name = "unpleasant_n"


### Usage example:

In [5]:
# Base para utilização do ollama com o llava

response = ollama.chat(
    model='llava', messages=[
  {
    'role': 'user',
    'content': 'Rate the hedonic valence of the following image, /home/jerre/git/StableDifussion_IC/images/Pleasant (+3)/Pleasant____landscape_S235711113_St25_G7.5.153.png, on a scale from 1 to 9, where 1 represents very negative and 9 represents very positive. Write only a number.',
  },
])
print(response['message']['content']) 

 8 


## Importing images and treating data:

In [6]:
# Importando e nomeando as imagens

image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp')
image_paths = []
image_names = []
image_seeds = []
image_groups = []

def import_images(directory, directory_name):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(image_extensions):
                image_paths.append(os.path.join(root, file))
                match = re.search(r'S(\d+)', file)
                if match:
                    seed = int(match.group(1))
                    image_names.append(f"{directory_name}_{(seed - 235711112)}")
                    image_seeds.append(seed)
                    image_groups.append(directory_name)
                else:
                    # Se não encontrar o padrão, atribui -1 como um valor padrão para seed
                    seed = -1
                    image_names.append(f"{directory_name}_unknown_seed")
                    image_seeds.append(seed)
                    image_groups.append(directory_name)

import_images(pleasant_dir, pleasant_dir_name)
import_images(unpleasant_dir, unpleasant_dir_name)
import_images(pleasant_p_dir, pleasant_p_dir_name)
import_images(unpleasant_p_dir, unpleasant_p_dir_name)
import_images(pleasant_n_dir, pleasant_n_dir_name)
import_images(unpleasant_n_dir, unpleasant_n_dir_name)

# Criando o DataFrame
df = pd.DataFrame({'path': image_paths, 'name': image_names, 'seed': image_seeds, 'group': image_groups})

In [7]:
df.head()

,path,name,seed,group
0,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_26,235711138,pleasant
1,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_24,235711136,pleasant
2,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_20,235711132,pleasant
3,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_1,235711113,pleasant
4,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_10,235711122,pleasant


In [8]:
# obtendo taxas de compressibilidade, tamanho de imagens

compressed_adress = "../images/Comprimida"
compression_quality = 95
if not os.path.exists(compressed_adress):
    os.makedirs(compressed_adress)

size_list = []
compressed_size_list = []
compressed_path_list = []
compression_ratio_list = []
complexity_list = []

for index, row in df.iterrows():
    image_path = row['path']
    image = Image.open(image_path)
    filename = os.path.basename(image_path)
    compressed_name = gen_hash(filename) + '.png'
    compressed_img_path = os.path.join(compressed_adress, compressed_name)
    image.save(compressed_img_path, optimize=True, quality=compression_quality)

    compressed_size = os.path.getsize(compressed_img_path)
    size = os.path.getsize(image_path)

    size_list.append(size)
    compressed_path_list.append(compressed_img_path)
    compressed_size_list.append(compressed_size)
    complexity_list.append(round((compressed_size/(512*512*3)), 2))

df['size'] = size_list
df['compressed_path'] = compressed_path_list
df['compressed_size'] = compressed_size_list
df['complexity'] = complexity_list

In [9]:
df.head()

,path,name,seed,group,size,compressed_path,compressed_size,complexity
0,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_26,235711138,pleasant,422997,../images/Comprimida/7ee30b4cf4b742cfca18ebaad...,406084,0.52
1,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_24,235711136,pleasant,490273,../images/Comprimida/55454d0f57d30a5853b446282...,474245,0.60
2,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_20,235711132,pleasant,569439,../images/Comprimida/24c53f2ee78c84c14aad6caac...,552196,0.70
3,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_1,235711113,pleasant,553613,../images/Comprimida/43d991a5a74354b7d4d0eb091...,536283,0.68
4,/home/jerre/git/Stable_Diffusion_IC/images/Ple...,pleasant_10,235711122,pleasant,460834,../images/Comprimida/de0749a1b5b4b4c4fbe40360e...,447861,0.57


# Single Pass:

In [10]:
'''
valence_list = []

for index, row in df.iterrows():
    image_path = row['compressed_path']
    
    with open(image_path, 'rb') as file:
        
        image= base64.b64encode(file.read()).decode('utf-8')
        
        response = ollama.chat(
            model='llava', messages=[
            {
            'role': 'system',
            'content': 'You are an average adult evaluating the hedonic valence of an image. You write nothing but a rating number.'
            },
            {
            'role': 'user',
            'content': 'Rate the hedonic valence of the image on a scale from 1 to 9, where 1 represents very negative and 9 represents very positive.',
            'image': [image]
            },
        ])
    
    valence_list.append(response['message']['content'])
    print(response['message']['content'])

df['valence1'] = valence_list
'''

"\nvalence_list = []\n\nfor index, row in df.iterrows():\n    image_path = row['compressed_path']\n    \n    with open(image_path, 'rb') as file:\n        \n        image= base64.b64encode(file.read()).decode('utf-8')\n        \n        response = ollama.chat(\n            model='llava', messages=[\n            {\n            'role': 'system',\n            'content': 'You are an average adult evaluating the hedonic valence of an image. You write nothing but a rating number.'\n            },\n            {\n            'role': 'user',\n            'content': 'Rate the hedonic valence of the image on a scale from 1 to 9, where 1 represents very negative and 9 represents very positive.',\n            'image': [image]\n            },\n        ])\n    \n    valence_list.append(response['message']['content'])\n    print(response['message']['content'])\n\ndf['valence1'] = valence_list\n"

# Double Pass:

#### Prompts para teste:

- user_prompt_double_pass: Rate the hedonic valence of the picture, on a scale from 1 to 9, where 1 means very negative and 9 means very positive

- system_prompt2: You are an adult participating in a study about affective content of pictures. Describe the picture in detail. Write the rating of the picture as a number on a scale from 1 to 9.

- system_parsing_prompt: You are a JSON parser. You must parse the given text only, and nothing else. Identify the rating expressed in the text and write it in JSON format: {'rating': number}.

- parsing_prompt: Parse the following text:

In [11]:
valence_list = []

for index, row in df.iterrows():
    image_path = row['compressed_path']
    
    with open(image_path, 'rb') as file:
        
        image= base64.b64encode(file.read()).decode('utf-8')
        
        response = ollama.chat(
            model='llava', messages=[
            {
            'role': 'system',
            'content': 'You are an average adult evaluating images. Feel free to express about them freely.'
            },
            {
            'role': 'user',
            'content': 'Describe the image and the emotions it conveys.',
            'image': [image]
            },
        ])
        
        content = response['message']['content']
        print(content)

        response = ollama.chat(
            model='llava', messages=[
            {
            'role': 'system',
            'content': 'You are just evaluating the hedonic valence of a text. You cant write anything but a rating number in the format "{"rating": "valence"}".'
            },
            {
            'role': 'user',
            'content': f'Rate the hedonic valence of the text: "{content}" on a scale from 1 to 9, where 1 represents very negative and 9 represents very positive.',
            },
        ])
    
        
    
    valence_list.append(get_json_value(response['message']['content']))
    print(response['message']['content'])
    print(get_json_value(response['message']['content']))

df['valence2'] = valence_list

 The image is a beautiful sunset with hues of orange, pink, and blue blending into the sky. It's serene and peaceful, evoking a sense of tranquility and calmness. The colors are warm and inviting, making me feel cozy and relaxed. The sight of the setting sun can also inspire contemplation or reflection on one's day. Overall, it's a visually pleasing image that I find very soothing and pleasant to look at. 
 {"rating": 8} 
8
 The image depicts a beautiful landscape with lush greenery, a tranquil lake, and majestic mountains in the background under a clear blue sky. The scene is peaceful, serene, and picturesque, evoking feelings of calmness and tranquility. It's a perfect representation of nature's beauty and the wonders it has to offer. I can imagine sitting by the lake, taking in this breathtaking view, and feeling connected to nature and the world around us. This image would likely bring a sense of relaxation and happiness to anyone who views it. 
 {"rating": 8} 
8
 Sure, I'd be happ

In [12]:
df.to_csv('data.csv', index=False)

In [18]:
df['valence2'] = pd.to_numeric(df['valence2'], errors='coerce')
media_por_grupo = df.groupby('group')['valence2'].mean()
print(media_por_grupo)

group
pleasant        6.666667
pleasant_n      6.866667
pleasant_p      6.100000
unpleasant      6.966667
unpleasant_n    6.333333
unpleasant_p    5.566667
Name: valence2, dtype: float64
